# PIERS Container BOL Data ETL 

This notebook builds an ETL pipeline for S&P Global's PIERS data. Data is extracted from CSV files downloaded from the Global Trade Analytics Suite, assigned appropriate datatypes, concatendated into a single dataframe, and loaded to an Apache Parquet file for storage.

In [1]:
#import libraries
import pandas as pd
import os
import time

#display settings
pd.set_option('display.max_columns', None)

## Extract and Transform

Read from csv into a pandas dataframe with appropriate dtypes

Note for future optimization: build a dictionary of column dtypes and assign within read_csv. 

In [2]:
def piers_imports_extractor(data):
    '''
    Extracts from downloaded PIERS csv files and performs initial cleaning 
    INPUT:
        data - str - the csv file to be extracted, including the path from current directory
    OUTPUT:
        df - pandas dataframe with appropriate column names and dtypes
    '''
    #read csv file 
    df = pd.read_csv(data, engine='pyarrow') # using the pyarrow engine engages more cpu cores 
    #unpack strings to list objects
    df['Container Number'] = df['Container Number'].str.split()
    df['Quantity of Commodity Short Description'] = df['Quantity of Commodity Short Description'].str.split(pat=';')
    df['Commodity Short Description'] = df['Commodity Short Description'].str.split(pat=',')
    #recast dates to datetime 
    df['Arrival Date'] = pd.to_datetime(df['Arrival Date'].astype(str), format='%Y%m%d') 
    #recast to int
    df['Quantity'] = pd.to_numeric(df['Quantity'], downcast='integer')
    #recast to categorical dtypes
    df[['Weight Unit', 'Quantity Type', 'Territory of Origin', 'Region of Origin', 'Port of Arrival Code', 'Port of Arrival',
        'Port of Departure Code', 'Port of Departure', 'Final Destination', 'Coastal Region', 'Clearing District', 'Place of Receipt',
        'Shipper', 'Carrier', 'SCAC', 'Mode of Transport']
        ] = df[['Weight Unit', 'Quantity Type', 'Territory of Origin', 'Region of Origin', 'Port of Arrival Code', 'Port of Arrival',
                'Port of Departure Code', 'Port of Departure', 'Final Destination', 'Coastal Region', 'Clearing District', 'Place of Receipt',
                'Shipper', 'Carrier', 'SCAC', 'Mode of Transport']].astype('category')
    return df    

In [3]:
#define path
path = 'data/raw/'
#get list of data files, ignoring any hidden files in directory 
datafiles = [file for file in os.listdir(path) if not file.startswith('.')]
#initialize dataframe
imports_df = pd.DataFrame()
#init filenumber
filenumber = 1

#extract from csv to clean dataframes and concat
print('Processing CSVs. \n There are {} files to process. \n'.format(len(datafiles)))
for filename in datafiles:
    start = time.time()
    print('Processing file number {}: {} ...'.format(filenumber, filename))
    file_df = piers_imports_extractor(path+filename)
    imports_df = pd.concat([imports_df, file_df])
    del file_df
    end = time.time()
    print('File number {} complete.'.format(filenumber))
    print('Processing this file took {} seconds.'.format(end-start))
    print('The dataframe is now {} GB.\n'.format(imports_df.memory_usage().sum()/1000000000))
    filenumber += 1
#recast to categorical dtypes
imports_df[
    ['Weight Unit', 'Quantity Type', 'Territory of Origin', 'Region of Origin', 'Port of Arrival Code', 'Port of Arrival',
    'Port of Departure Code', 'Port of Departure', 'Final Destination', 'Coastal Region', 'Clearing District', 'Place of Receipt',
    'Shipper', 'Carrier', 'SCAC', 'Mode of Transport']
    ] = imports_df[
        ['Weight Unit', 'Quantity Type', 'Territory of Origin', 'Region of Origin', 'Port of Arrival Code', 'Port of Arrival',
            'Port of Departure Code', 'Port of Departure', 'Final Destination', 'Coastal Region', 'Clearing District', 'Place of Receipt',
            'Shipper', 'Carrier', 'SCAC', 'Mode of Transport']].astype('category')

Processing CSVs. 
 There are 25 files to process. 

Processing file number 1: PIERS import records 2022-09-01 to 2023-01- 31 CD439702FC014C1E964C1F770095E3E9.csv ...
File number 1 complete.
Processing this file took 15.416409015655518 seconds.
The dataframe is now 1.333442503 GB.

Processing file number 2: PIERS import records 2021-01-01 to 2021-06-30 885FF5029B5F4A7D9984B64AC1CAC155.csv ...
File number 2 complete.
Processing this file took 21.673197746276855 seconds.
The dataframe is now 3.66803493 GB.

Processing file number 3: PIERS import records 2023-09-06 to 2023-11- 31 63E23DF3FF4C45D396DDF74608618685.csv ...
File number 3 complete.
Processing this file took 11.38774037361145 seconds.
The dataframe is now 4.583217962 GB.

Processing file number 4: PIERS import records 2019 01-06 DDE7D8AFA5C540B9BEF53A673D284078.csv ...
File number 4 complete.
Processing this file took 17.836590051651 seconds.
The dataframe is now 6.401839483 GB.

Processing file number 5: PIERS import records 20

/Users/adamwilson/miniconda3/envs/wsu/lib/python3.12/site-packages/pandas/core/dtypes/cast.py:375: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)


File number 7 complete.
Processing this file took 22.235116004943848 seconds.
The dataframe is now 11.537392425 GB.

Processing file number 8: PIERS import records 2022-03-01 to 2022-08- 31 DC6C0A9417E1411796365E8CB68EA104.csv ...
File number 8 complete.
Processing this file took 34.023886919021606 seconds.
The dataframe is now 14.23123544 GB.

Processing file number 9: PIERS import records 2016 07-12 0E1786F127724FC7840C3C221E3AB4EC.csv ...
File number 9 complete.
Processing this file took 27.09745216369629 seconds.
The dataframe is now 15.927301924 GB.

Processing file number 10: PIERS import records 2020 07-12 F4590742040E42B4A8945821EE4526BC.csv ...
File number 10 complete.
Processing this file took 46.35431909561157 seconds.
The dataframe is now 18.054247048 GB.

Processing file number 11: PIERS import records 2013 07-12 AAD497A1356E480EA50A63677D75192D.csv ...
File number 11 complete.
Processing this file took 35.93884301185608 seconds.
The dataframe is now 19.654800952 GB.

Proc

/var/folders/dc/sgfd2dls28n4y4v4_jw03ly40000gp/T/ipykernel_16062/4265692848.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imports_df = pd.concat([imports_df, file_df], ignore_index=True)


File number 14 complete.
Processing this file took 52.8095760345459 seconds.
The dataframe is now 24.17147838 GB.

Processing file number 15: PIERS import records 2022 01-02 8C26A4D120BD40FD9682336FAD5E0FA6.csv ...
File number 15 complete.
Processing this file took 50.39344000816345 seconds.
The dataframe is now 24.967987664 GB.

Processing file number 16: PIERS import records 2021-07-01 to 2021-12-31 2594E636259E4E6D9E98DB25CBD3D4C8.csv ...
File number 16 complete.
Processing this file took 75.01745176315308 seconds.
The dataframe is now 27.297890424 GB.

Processing file number 17: PIERS import records 2023-02-01 to 2023-09-05 CB5380DE33CB4169A676AD839290E38E.csv ...
File number 17 complete.
Processing this file took 968.1512818336487 seconds.
The dataframe is now 29.648141688 GB.

Processing file number 18: PIERS import records 2014 01-06 3D15D62D5DCC4DC0B185D2A13DA5AF19.csv ...
File number 18 complete.
Processing this file took 131.6448621749878 seconds.
The dataframe is now 31.1924

In [4]:
# inspect output 
display(imports_df.head())
imports_df.info()

,Weight,Weight Unit,Quantity,Quantity Type,TEUs,Estimated Value,Arrival Date,Container Piece Count,Quantity of Commodity Short Description,Territory of Origin,Region of Origin,Port of Arrival Code,Port of Arrival,Port of Departure Code,Port of Departure,Final Destination,Coastal Region,Clearing District,Place of Receipt,Shipper,Shipper Address,Consignee,Consignee Address,Notify Party,Notify Party Address,Also Notify Party,Also Notify Party Address,Raw Commodity Description,Marks Container Number,Marks Description,HS Code,JOC Code,Commodity Short Description,Container Number,Carrier,SCAC,Vessel Name,Voyage Number,Pre Carrier,IMO Number,Inbond Code,Mode of Transport,Bill of Lading Number
0,9384.0,KG,920,PCS,2.00,26087.0,2023-01-31,1,[920],CHINA (MAINLAND),NORTH EAST ASIA,5.0,VANCOUVER BC,58023.0,BUSAN,,WEST,NaN,QINGDAO,QINGDAO TIANYI NO 11 WOODWORK,None,JYSK LINEN N FURNITURE,25 KING EDWARD ST,None,None,None,None,WOODEN JEWELRY CABINET ORDER NUMBER CD 124 528,DESCRIPTION PRODUCT DIMENSIONCOLOR MATERIAL N...,40,940350,7275000,"[FURNITURE, FIXTURES; NOS (* 7275)]",[MEDU9352303],MEDITERRANEAN SHIPPING COMPANY,MEDU,MSC DARWIN VI,UK251,NaN,9200689.0,NaN,MARITIME,SHKK156975611601
1,11416.0,KG,583,CTN,1.00,30138.0,2023-01-31,1,[583],CHINA (MAINLAND),NORTH EAST ASIA,5.0,VANCOUVER BC,58023.0,BUSAN,,WEST,NaN,QINGDAO,YANTAI HONGTIAN AUTOPARTS,None,UNI SELECT USA,8039 5TH LINE,None,None,None,None,BRAKE ROTOR 583CTNS 20PLTS,None,20,870830,6922000,"[PARTS; AUTOMOBILE, MOTOR VECHILES]",[MEDU1232702],MEDITERRANEAN SHIPPING COMPANY,MEDU,MSC DARWIN VI,UK251,NaN,9200689.0,NaN,MARITIME,SHKK156976599151
2,21487.0,KG,1500,PKG,2.00,71121.0,2023-01-31,1,[1500],AUSTRALIA,OCEANIA,4601.0,NEW YORK,22519.0,COLON,,EAST,NaN,MELBOURNE,TRIMBOLI FAMILY WINES,None,LIONSTONE INTERNATIONAL,28188 N. BALLARD DRIVE,JF HILLEBRAND USA CUSTOMS DESK RAH,None,None,None,WINE NOT SPARKLING GRAPE MUST WITH FERMT.PRE .,VT.BY ALCOHOL IN 2 L CONT.NO MARKS,40,2204,1673000,[WINE; NOS (* 1671/1674)],[TGHU5172870],CMA-CGM,CMDU,MARFRET GUYANE,0PPTX,NaN,9362334.0,NaN,MARITIME,BQEGAUA279294
3,94080.0,KG,5880,PKG,10.00,311400.0,2023-01-31,5,"[1176, 1176, 1176, 1176, 1176]",NEW ZEALAND,OCEANIA,4601.0,NEW YORK,24128.0,KINGSTON,,EAST,NaN,NELSON,VINLINK MARLBOROUGH,None,DC FLYNT MW SELECTIONS,902 S. DIVISION ST,DC FLYNT MW SELECTIONS,None,None,None,WINE NOT SPARKLING GRAPE MUST WITH FERMT.PRE ....,VT.BY ALCOHOL IN 2 L CONT.NO MARKS VT.BY ALCOH...,40; 40; 40; 40; 40,2204,1673000,[WINE; NOS (* 1671/1674)],"[TEMU8382775, GAOU6853790, CMAU6652090, CMAU64...",CMA-CGM,CMDU,MARFRET GUYANE,0UADG,NaN,9362334.0,NaN,MARITIME,BQEGUSR019686
4,107100.0,KG,75,DRS,8.75,761480.0,2023-01-31,5,"[15, 15, 15, 15, 15]",CHINA (MAINLAND),NORTH EAST ASIA,5.0,VANCOUVER BC,57035.0,SHANGHAI,,WEST,NaN,SHANGHAI,ZHEJIANG XINAN CHEMICAL INDUSTRIAL,None,JET AIR FERRARA,None,L E I,456 HUMBER PLACE,None,None,GLYPHOSATE 540G L SL PRESENT AS GLYPHOSATE PO ...,TASSIUM SALTNM TASSIUM SALTNM TASSIUM SALTNM T...,None,380893,4051530,[INSECTICIDE],"[MSDU2444663, MSDU1536852, MEDU3857221, FTAU14...",MEDITERRANEAN SHIPPING COMPANY,MEDU,MSC VANESSA,251A,NaN,9251688.0,NaN,MARITIME,MOSJSZXS042667


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136178647 entries, 0 to 136178646
Data columns (total 43 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   Weight                                   float64       
 1   Weight Unit                              category      
 2   Quantity                                 int64         
 3   Quantity Type                            category      
 4   TEUs                                     float64       
 5   Estimated Value                          float64       
 6   Arrival Date                             datetime64[ns]
 7   Container Piece Count                    int64         
 8   Quantity of Commodity Short Description  object        
 9   Territory of Origin                      category      
 10  Region of Origin                         category      
 11  Port of Arrival Code                     category      
 12  Port of Arrival         

## Load

In [5]:
#save to parquet file
imports_df.to_parquet('data/piers_imports.parquet', index=False, engine='fastparquet') #requires fastparquet dependency  

#delete imports df
del imports_df